In [ ]:
import re
import json
from timeout_decorator import timeout

In [ ]:
name = "Text_DaVinci_Refined_Output"
filename = f'../Generation/{name}.json'
outfilename = f'./{name}_Compiled_Result.json'

In [ ]:
with open(filename, 'r') as f:
    data = json.load(f)
len(data)

In [ ]:
@timeout(60)
def match_expression(expression, text):
    try:
        return re.search(expression, text) is not None
    except Exception as e:
        print(expression,": ", e)
        return None

In [ ]:
compilation_result = []
for item in data:
    print("Starting item: ", item['id'])
    id = item['id']
    matches = item['matches']
    non_matches = item['non_matches']
    for completion in item['gpt3.5_output']['choices']:
        expression = completion['message']['content']
        item['completion'] = expression
        is_valid = True
        for match in matches:
            for match in matches:
                try:
                    is_match = match_expression(expression, match)
                except TimeoutError:
                    print("TimeoutError: ", expression, match)
                    is_match = None
                if is_match is None or is_match == False:
                    is_valid = False
                    break
        for non_match in non_matches:
            try:
                is_match = match_expression(expression, non_match)
            except TimeoutError:
                print("TimeoutError: ", expression, match)
                is_match = None
            if is_match is None or is_match == True:
                is_valid = False
                break
        item['passed'] = is_valid
        compilation_result.append({
            'id': id,
            'completion': expression,
            'passed': is_valid
        })  
    print("Ending item: ", item['id'])

In [ ]:
with open(outfilename, 'w') as f:
    json.dump(compilation_result, f, indent=4)